In [ ]:
%load_ext line_profiler

In [ ]:
import os
import json
import solcx
import solcast
import re
import pandas as pd
from solidity_parser import parser
from tqdm import tqdm

In [ ]:

MAPPING = {
    'reentrancy': {
        "slither": [
            "reentrancy-benign",
            "reentrancy-events",
            "reentrancy-no-eth",
            "reentrancy-unlimited-gas",
            "reentrancy-eth",
        ],
        'semgrep': [
            "compound-borrowfresh-reentrancy",
            "erc721-reentrancy",
            "curve-readonly-reentrancy",
            "erc777-reentrancy",
            "erc677-reentrancy"
        ],
        'smartcheck': [

        ]
    },
    'low-level-calls': {
        'slither': [],
        'semgrep': [
            'arbitrary-low-level-call'
        ],
        'smartcheck': [
            'SOLIDITY_CALL_WITHOUT_DATA'
        ]
    },
    'access-control': {
        'slither': [
            "arbitrary-send-erc20",
            "arbitrary-send-eth",
            "suicidal",
            "arbitrary-send-erc20-permit",
        ],
        'semgrep': [
            'compound-sweeptoken-not-restricted',
            'erc20-public-burn',
            'accessible-selfdestruct',
            'oracle-price-update-not-restricted',
            'uniswap-callback-not-protected',

        ],
        'smartcheck': [
            'SOLIDITY_TX_ORIGIN'
        ]
    },
    'delegation': {
        'slither': [
            "controlled-delegatecall",
            "delegatecall-loop",
        ],
        'semgrep': [
            'delegatecall-to-arbitrary-address',

        ],
        'smartcheck': [

        ]
    },
    'arithmetic': {
        'slither': [
            "divide-before-multiply",
        ],
        'semgrep': [
            'basic-arithmetic-underflow'
        ],
        'smartcheck': [
            'SOLIDITY_DIV_MUL'
        ]
    },
    'oracle-manipulation': {
        'slither': [],
        'semgrep': [
            'keeper-network-oracle-manipulation'
        ],
        'smartcheck': [

        ]
    },
    'input-validation': {
        'slither': [
            "missing-zero-check",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'shadowing': {
        'slither': [
            "shadowing-local",
            "shadowing-state",
            "shadowing-abstract",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'compliance': {
        'slither': [
            "erc20-interface",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ERC20_TRANSFER_SHOULD_THROW'
        ]
    },
    'timestamp': {
        "slither": [
            "timestamp",
            "weak-prng",
        ],
        'semgrep': [

        ],
        'smartcheck': [
        ]
    },
    'initialization': {
        'slither': [
            "uninitialized-local",
            "uninitialized-state",
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'poor-logic-flaws': {
        "slither": [
            "incorrect-equality",
            "boolean-cst",
        ],
        'semgrep': [
            'incorrect-use-of-blockhash'
        ],
        'smartcheck': [
            'SOLIDITY_EXACT_TIME',
            'SOLIDITY_BALANCE_EQUALITY'
        ]
    },
    'denial-of-service': {
        "slither": [
            "locked-ether",
            "calls-loop",
            "msg-value-loop",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_LOCKED_MONEY',
            'SOLIDITY_TRANSFER_IN_LOOP'
        ]
    },
    'state-corruption': {
        "slither": [
            "controlled-array-length",
        ],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ARRAY_LENGTH_MANIPULATION'
        ]
    },
    'function-behavior': {
        "slither": [
            'incorrect-modifier'
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'external-call-validation': {
        "slither": [
            "unchecked-transfer",
            "unchecked-lowlevel"
        ],
        'semgrep': [

        ],
        'smartcheck': [

        ]
    },
    'front-running': {
        "slither": [],
        'semgrep': [

        ],
        'smartcheck': [
            'SOLIDITY_ERC20_APPROVE'
        ]
    }
}

REVERSE_MAPPING = {
    'reentrancy-benign': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-events': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-no-eth': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-unlimited-gas': {'category': 'reentrancy', 'analyzer': 'slither'},
    'reentrancy-eth': {'category': 'reentrancy', 'analyzer': 'slither'},
    'compound-borrowfresh-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc721-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'curve-readonly-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc777-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'erc677-reentrancy': {'category': 'reentrancy', 'analyzer': 'semgrep'},
    'arbitrary-low-level-call': {'category': 'low-level-calls', 'analyzer': 'semgrep'},
    'SOLIDITY_CALL_WITHOUT_DATA': {'category': 'low-level-calls', 'analyzer': 'smartcheck'},
    'arbitrary-send-erc20': {'category': 'access-control', 'analyzer': 'slither'},
    'arbitrary-send-eth': {'category': 'access-control', 'analyzer': 'slither'},
    'suicidal': {'category': 'access-control', 'analyzer': 'slither'},
    'arbitrary-send-erc20-permit': {'category': 'access-control', 'analyzer': 'slither'},
    'compound-sweeptoken-not-restricted': {'category': 'access-control', 'analyzer': 'semgrep'},
    'erc20-public-burn': {'category': 'access-control', 'analyzer': 'semgrep'},
    'accessible-selfdestruct': {'category': 'access-control', 'analyzer': 'semgrep'},
    'oracle-price-update-not-restricted': {'category': 'access-control', 'analyzer': 'semgrep'},
    'uniswap-callback-not-protected': {'category': 'access-control', 'analyzer': 'semgrep'},
    'SOLIDITY_TX_ORIGIN': {'category': 'access-control', 'analyzer': 'smartcheck'},
    'controlled-delegatecall': {'category': 'delegation', 'analyzer': 'slither'},
    'delegatecall-loop': {'category': 'delegation', 'analyzer': 'slither'},
    'delegatecall-to-arbitrary-address': {'category': 'delegation', 'analyzer': 'semgrep'},
    'divide-before-multiply': {'category': 'arithmetic', 'analyzer': 'slither'},
    'basic-arithmetic-underflow': {'category': 'arithmetic', 'analyzer': 'semgrep'},
    'SOLIDITY_DIV_MUL': {'category': 'arithmetic', 'analyzer': 'smartcheck'},
    'keeper-network-oracle-manipulation': {'category': 'oracle-manipulation', 'analyzer': 'semgrep'},
    'missing-zero-check': {'category': 'input-validation', 'analyzer': 'slither'},
    'shadowing-local': {'category': 'shadowing', 'analyzer': 'slither'},
    'shadowing-state': {'category': 'shadowing', 'analyzer': 'slither'},
    'shadowing-abstract': {'category': 'shadowing', 'analyzer': 'slither'},
    'erc20-interface': {'category': 'compliance', 'analyzer': 'slither'},
    'SOLIDITY_ERC20_TRANSFER_SHOULD_THROW': {'category': 'compliance', 'analyzer': 'smartcheck'},
    'timestamp': {'category': 'timestamp', 'analyzer': 'slither'},
    'weak-prng': {'category': 'timestamp', 'analyzer': 'slither'},
    'uninitialized-local': {'category': 'initialization', 'analyzer': 'slither'},
    'uninitialized-state': {'category': 'initialization', 'analyzer': 'slither'},
    'incorrect-equality': {'category': 'poor-logic-flaws', 'analyzer': 'slither'},
    'boolean-cst': {'category': 'poor-logic-flaws', 'analyzer': 'slither'},
    'incorrect-use-of-blockhash': {'category': 'poor-logic-flaws', 'analyzer': 'semgrep'},
    'SOLIDITY_EXACT_TIME': {'category': 'poor-logic-flaws', 'analyzer': 'smartcheck'},
    'SOLIDITY_BALANCE_EQUALITY': {'category': 'poor-logic-flaws', 'analyzer': 'smartcheck'},
    'locked-ether': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'calls-loop': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'msg-value-loop': {'category': 'denial-of-service', 'analyzer': 'slither'},
    'SOLIDITY_LOCKED_MONEY': {'category': 'denial-of-service', 'analyzer': 'smartcheck'},
    'SOLIDITY_TRANSFER_IN_LOOP': {'category': 'denial-of-service', 'analyzer': 'smartcheck'},
    'controlled-array-length': {'category': 'state-corruption', 'analyzer': 'slither'},
    'SOLIDITY_ARRAY_LENGTH_MANIPULATION': {'category': 'state-corruption', 'analyzer': 'smartcheck'},
    'incorrect-modifier': {'category': 'function-behavior', 'analyzer': 'slither'},
    'unchecked-transfer': {'category': 'external-call-validation', 'analyzer': 'slither'},
    'unchecked-lowlevel': {'category': 'external-call-validation', 'analyzer': 'slither'},
    'SOLIDITY_ERC20_APPROVE': {'category': 'front-running', 'analyzer': 'smartcheck'}
}

In [ ]:
available_versions = [
                "0.4.12",
                "0.4.14",
                "0.4.15",
                "0.4.16",
                "0.4.18",
                "0.4.19",
                "0.4.21",
                "0.4.23",
                "0.4.24",
                "0.4.25",
                "0.4.26",
                "0.5.0",
                "0.5.2",
                "0.5.3",
                "0.5.4",
                "0.5.5",
                "0.5.8",
                "0.5.9",
                "0.5.10",
                "0.5.11",
                "0.5.12",
                "0.5.13",
                "0.5.14",
                "0.5.15",
                "0.5.16",
                "0.5.17",
                "0.6.0",
                "0.6.2",
                "0.6.6",
                "0.6.8",
                "0.6.11",
                "0.6.12",
                "0.6.9",
                "0.7.0",
                "0.7.1",
                "0.7.3",
                "0.7.4",
                "0.7.5",
                "0.7.6",
                "0.8.1",
                "0.8.2",
                "0.8.3",
                "0.8.4",
                "0.8.5",
                "0.8.6",
                "0.8.7",
                "0.8.8",
                "0.8.9",
                "0.8.10",
                "0.8.11",
                "0.8.12",
                "0.8.13"
            ]

In [ ]:
dataset_name = 'compilable_DAppSCAN'
ast_dataset_name = 'ast_DAppSCAN'
failed_analysis_files_name = 'failed_analysis_files.txt'


In [ ]:
files_not_having_ast = open('./files_not_having_ast.txt', 'r').read().split('\n')
failed_analysis_files = open('./failed_analysis_files_name.txt', 'w')
failed_analysis_files_list = list()
success_analysis_files_list = list()
file_not_found_list = list()

In [ ]:

statistics_matrices = {
    category: pd.DataFrame(
        index=pd.Index(detectors, name="detector"),
        columns=detectors,
        data=0
    )
    for category, analyzers in MAPPING.items()
    for detectors in [[detector for analyzer_detectors in analyzers.values() for detector in analyzer_detectors]]
}


In [ ]:

def get_solidity_compiler_version(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # If there are matches, sort them and return the oldest (smallest) version number
    if version_numbers:
        oldest_version = sorted(version_numbers)[0]  # Sorts and picks the oldest version
        return '.'.join(map(str, oldest_version))  # Converts tuple back to string format
    else:
        return None

def get_solidity_compiler_version_and_clean_code(solidity_code):
    # Regular expression pattern to capture only the version number part
    pattern_version = r"pragma\s+solidity\s+[^\d]*(\d+\.\d+\.\d+);"
    # Pattern to match entire pragma statements
    pattern_full = r"(pragma\s+solidity\s+[^\d]*\d+\.\d+\.\d+;)"

    # Find all version numbers in the solidity code
    matches = re.findall(pattern_version, solidity_code)

    # Convert version numbers to tuples of integers for easy comparison
    version_numbers = [tuple(map(int, match.split('.'))) for match in matches]

    # Initialize modified code as the original solidity code
    modified_code = solidity_code

    # If there are matches, find the oldest version
    if version_numbers:
        oldest_version = sorted(version_numbers)[0]  # Sorts and picks the oldest version
        oldest_version_str = '.'.join(map(str, oldest_version))  # Converts tuple back to string format
        
        # Remove all pragma statements
        modified_code = re.sub(pattern_full, '', modified_code)
        
        # Add back the oldest pragma statement at the beginning
        modified_code = f"pragma solidity ^{oldest_version_str};\n" + modified_code.strip()

        return oldest_version_str, modified_code
    else:
        return None, solidity_code


def compile_solidity_contract_new(file_path):
    output_json = None 
    source_code = open(file_path, 'r').read()

    input_json = {
            'language': 'Solidity',
            'sources': {file_path: {'content': source_code}},
            'settings': {
                'outputSelection': {
                    '*': {
                        '*': ['metadata', 'evm.bytecode', 'evm.bytecode.sourceMap', 'abi'],
                        '': ['ast']  # Requesting AST output here
                    }
                }
            }
        }
    
    try:
        with open(file_path, 'r') as fp:
            content = fp.read()
            pragma = re.search(r'pragma solidity.*', content, re.MULTILINE).group(0)

            if pragma:

                try:
                    solcx.set_solc_version_pragma(pragma)
                    with open(file_path, 'r') as f:
                        content = f.read()

                    # result = solcx.compile_source(content, 
                    #                      output_values=["ast"],
                    #                      overwrite=True,
                    #                      allow_empty=True
                    #                      )
                    output_json = solcx.compile_standard(input_json)
                    success_analysis_files_list.append(success_analysis_files_list)

                except Exception as e:
                    print(f'File: {file_path}')
                    print(f'Error setting solc version to {pragma}: {e}')
                    failed_analysis_files_list.append(file_path)
                    
    except Exception as e:
        print(f'Error getting solc version for {file_path}: {e}')
        failed_analysis_files.append(file_path)
        return None

    
    return output_json




def compile_solidity_contract(file_path):
    output_json = None 
    source_code = open(file_path, 'r').read()

    input_json = {
            'language': 'Solidity',
            'sources': {file_path: {'content': source_code}},
            'settings': {
                'outputSelection': {
                    '*': {
                        '*': ['metadata', 'evm.bytecode', 'evm.bytecode.sourceMap', 'abi'],
                        '': ['ast']  # Requesting AST output here
                    }
                }
            }
        }
    
    try:
        with open(file_path, 'r') as fp:
            content = fp.read()
            pragma = re.search(r'pragma solidity.*', content, re.MULTILINE).group(0)

            if pragma:

                try:
                    solcx.set_solc_version_pragma(pragma)
                    with open(file_path, 'r') as f:
                        content = f.read()

                    # result = solcx.compile_source(content, 
                    #                      output_values=["ast"],
                    #                      overwrite=True,
                    #                      allow_empty=True
                    #                      )
                    output_json = solcx.compile_standard(input_json)
                    success_analysis_files_list.append(file_path)
                    
                except Exception as e:
                    #print(f'File: {file_path}')
                    # print(f'Error setting solc version to {pragma}: {e}')
                    failed_analysis_files_list.append(file_path)
                    
    except Exception as e:
        print(f'Error getting solc version for {file_path}: {e}')
        failed_analysis_files.append(file_path)
        return None

    
    return output_json


def print_contract_ast(compiled_sol):
    nodes = solcast.from_standard_output(compiled_sol)
    for node in nodes:
        print(node)
        # Example to explore further: print the names of contracts in a SourceUnit
        for child in node['nodes']:
            if child['nodeType'] == 'ContractDefinition':
                print(f"Contract Name: {child['name']}")


def get_enclosing_function_and_contract(output_json, solidity_filename, start_line, end_line, root_node, source_code):

    function = None
    contract = None

    
    lines = source_code.split("\n")
    start_index = sum(len(lines[i]) + 1 for i in range(start_line - 1))
    start_index_adjusted = start_index + \
        len(lines[start_line - 1]) - len(lines[start_line - 1].lstrip())
    end_index = start_index + \
        sum(len(lines[i]) + 1 for i in range(start_line - 1, end_line)) - 1

    result = root_node.children(
        required_offset=(start_index_adjusted, end_index))

    for node in result:
        if node.nodeType == 'ContractDefinition':
            contract = {
                "offset": node.offset,
                "name": node.name
            }
        elif node.nodeType == 'FunctionDefinition':
            function = {
                "offset": node.offset,
                "name": node.name
            }

    return {
        "contract": contract,
        "function": function
    }


def count_json_files_recursively(start_path='.'):
    json_file_count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if file.endswith('.json'):
                json_file_count += 1
    return json_file_count


In [121]:

def update_statistics(findings_for_file):
    global statistics_matrices
    for detector, findings in findings_for_file.items():
        for finding in findings:
            for other_detector, other_findings in findings_for_file.items():

                if other_detector == detector or REVERSE_MAPPING[detector]['category'] != REVERSE_MAPPING[other_detector]['category']:
                    continue
                for other_finding in other_findings:
                    if finding['function'] == other_finding['function']:
                        statistics_matrices[f"{REVERSE_MAPPING[detector]['category']}"][f"{detector}"][f"{other_detector}"] += 1


def process_json_file(json_file_path):
    with open(json_file_path, 'r') as file:
        data = json.load(file)

    # Dictionary to hold the processed information
    processed_info = {}

    # Loop through each file in the JSON
    for file_name, analysis_info in data.items():
        file_details = {}
        # Loop through each detector for the current file
        for detector, detector_info in analysis_info.items():
            # Check if the analysis was successful and vulnerability findings are not empty
            if detector_info.get("successfull_analysis", False) and detector_info.get("vulnerability_findings"):
                file_details[detector] = detector_info["vulnerability_findings"]

        # Only add details for this file if there are any non-empty findings
        if file_details:
            processed_info[file_name] = file_details

    # At this point, processed_info contains the extracted data without empty findings
    # You can process this data further as needed
    #print(f"Processed vulnerabilties.json file of the following project => {json_file_path}:")
    # project_count = 0
    # for file, file_information in processed_info.items():
    #     project_count += 1
    #     relative_file_path = f"./{dataset_name}{json_file_path[17:-21]}/{file}"
    #     relative_ast_file_path = f"./{ast_dataset_name}{json_file_path[17:-21]}/{file[:-3]}json"
        
    #     output_json = compile_solidity_contract(relative_file_path)
    #     if output_json is None or relative_file_path in files_not_having_ast:
    #         # file is not compilable; just pass to the next one.
    #         continue

    #     #nodes = solcast.from_standard_output(output_json)
    #     root_node = solcast.from_ast(output_json["sources"][relative_file_path]["ast"])
    
    project_count = 0
    for file, file_information in tqdm(processed_info.items(), desc="Processing Files"):
        project_count += 1
        relative_file_path = f"./{dataset_name}{json_file_path[17:-21]}/{file}"
        relative_ast_file_path = f"./{ast_dataset_name}{json_file_path[17:-21]}/{file[:-3]}json"
        
        output_json = compile_solidity_contract(relative_file_path)
        if output_json is None or relative_file_path in files_not_having_ast:
            # file is not compilable; just pass to the next one.
            continue

        #nodes = solcast.from_standard_output(output_json)
        root_node = solcast.from_ast(output_json["sources"][relative_file_path]["ast"])

        ast_string = open(relative_ast_file_path, 'r').read() 
        ast_dict = json.loads(ast_string)
        new_ast = {
            "absolutePath": relative_file_path,
        }
        for key, info in ast_dict.items():
            new_ast

        # open('compile_ast.json', 'w').write(json.dumps(output_json["sources"][relative_file_path]["ast"]))
        # open('ready_ast.json', 'w').write(json.dumps(ast_json))

        # root_node = solcast.from_ast(ast_json)
        source_code = open(relative_file_path, 'r').read()
        
        findings_for_file = dict()
        for analyzer, analysis_results in file_information.items():
            # print(f"Analyzer is: {analyzer};")
            # print(analysis_results)
            for i in range(len(analysis_results)):
                vuln_start_line = analysis_results[i]["vulnerability_from_line"]
                vuln_end_line = analysis_results[i]["vulnerability_to_line"]
                vulnerability_name = analysis_results[i]["name"]
                # print(
                #    f"## Vulnerability is: {vulnerability_name}; vulnerability start line is: {vuln_start_line}; end line is: {vuln_end_line}")
                # print("* * * * * * * * * * * * * * * * * *")
                # print(
                #    f"start line: {vuln_start_line}, end line: {vuln_end_line}")
                if vuln_start_line == None:
                    if vuln_end_line == None:
                        break
                    else:
                        vuln_start_line = vuln_end_line
                elif vuln_end_line == None:
                    vuln_end_line = vuln_start_line

                # commenting for now, as I am doing another part of the code:
                    
                

                result = get_enclosing_function_and_contract(
                    output_json, relative_file_path, vuln_start_line, vuln_end_line, root_node, source_code)
                    
                if vulnerability_name not in REVERSE_MAPPING:
                    continue
                # else:
                #     print(
                #         f">>> The {vulnerability_name} detector name is indeed in reverse mapping <<<")
                if vulnerability_name in findings_for_file:
                    findings_for_file[f"{vulnerability_name}"].append({
                        'function': result['function']['name'] if result['function'] is not None else result['function'],
                        'contract': result['contract']['name'] if result['contract'] is not None else result['contract'],
                    })
                else:
                    findings_for_file.update({
                        f"{vulnerability_name}": [{
                            'function': result['function']['name'] if result['function'] is not None else result['function'],
                            'contract': result['contract']['name'] if result['contract'] is not None else result['contract'],
                        },]
                    })
                # print(
                #    f"Contract is: {result['contract']}, Function is: {result['function']}")
        
        if findings_for_file:
            #print(f"For file {file}; findings are: ")
            #print(findings_for_file)
            update_statistics(findings_for_file)
            #print('# # # # # # # # # # # # # # # # # #')
    #print(f"----------------------------------------- Project #{project_count} finished, moving to the next one ------------------------------------------ ")

    # For demonstration, just returning the processed data

    return processed_info


def read_and_process_json_files(start_directory):
    for root, dirs, files in os.walk(start_directory):
        for file in files:
            if file == 'vulnerabilities.json':
                json_file_path = os.path.join(root, file)
                # Process the JSON file
                try:
                    processed_data = process_json_file(json_file_path)
                except FileNotFoundError:
                    file_not_found_list.append(json_file_path)
                
                # Add your processing code here, using the 'processed_data' variable



In [122]:
read_and_process_json_files('.')


print('\n ------------------------------ All supposed to be finished by this point ----------------------------------')


Processing Files:   0%|          | 0/5 [00:00<?, ?it/s]
Processing Files: 0it [00:00, ?it/s]
Processing Files:  12%|█▏        | 7/60 [00:01<00:10,  5.18it/s]
Processing Files: 0it [00:00, ?it/s]
Processing Files: 100%|██████████| 1/1 [00:00<00:00,  4.50it/s]
Processing Files: 0it [00:00, ?it/s]
Processing Files:  11%|█         | 8/76 [00:45<06:25,  5.67s/it]
Processing Files: 0it [00:00, ?it/s]
Processing Files: 100%|██████████| 1/1 [00:01<00:00,  1.21s/it]
Processing Files: 0it [00:00, ?it/s]
Processing Files: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]
Processing Files: 0it [00:00, ?it/s]
Processing Files:   6%|▋         | 1/16 [00:00<00:10,  1.50it/s]


 ------------------------------ All supposed to be finished by this point ----------------------------------


In [125]:
len(success_analysis_files_list)
len(failed_analysis_files_list)

2904

In [ ]:
%lprun -f read_and_process_json_files -f process_json_file -f get_enclosing_function_and_contract read_and_process_json_files('.')
#%lprun -f get_enclosing_function_and_contract read_and_process_json_files('.')
# %lprun -f read_and_process_json_files -f compile_solidity_contract read_and_process_json_files('.')

In [126]:
statistics_matrices['reentrancy']

,reentrancy-benign,reentrancy-events,reentrancy-no-eth,reentrancy-unlimited-gas,reentrancy-eth,compound-borrowfresh-reentrancy,erc721-reentrancy,curve-readonly-reentrancy,erc777-reentrancy,erc677-reentrancy
detector,,,,,,,,,,
reentrancy-benign,0,0,870,0,0,33,0,0,0,0
reentrancy-events,0,0,0,0,0,0,0,0,0,0
reentrancy-no-eth,870,0,0,0,0,0,0,0,0,0
reentrancy-unlimited-gas,0,0,0,0,0,0,0,0,0,0
reentrancy-eth,0,0,0,0,0,0,0,0,0,0
compound-borrowfresh-reentrancy,33,0,0,0,0,0,0,0,0,0
erc721-reentrancy,0,0,0,0,0,0,0,0,0,0
curve-readonly-reentrancy,0,0,0,0,0,0,0,0,0,0
erc777-reentrancy,0,0,0,0,0,0,0,0,0,0


In [127]:
statistics_matrices['low-level-calls']

,arbitrary-low-level-call,SOLIDITY_CALL_WITHOUT_DATA
detector,,
arbitrary-low-level-call,0,0
SOLIDITY_CALL_WITHOUT_DATA,0,0


In [128]:
statistics_matrices['access-control']

,arbitrary-send-erc20,arbitrary-send-eth,suicidal,arbitrary-send-erc20-permit,compound-sweeptoken-not-restricted,erc20-public-burn,accessible-selfdestruct,oracle-price-update-not-restricted,uniswap-callback-not-protected,SOLIDITY_TX_ORIGIN
detector,,,,,,,,,,
arbitrary-send-erc20,0,0,0,0,0,0,0,0,0,0
arbitrary-send-eth,0,0,0,0,0,0,0,0,0,0
suicidal,0,0,0,0,0,0,0,0,0,0
arbitrary-send-erc20-permit,0,0,0,0,0,0,0,0,0,0
compound-sweeptoken-not-restricted,0,0,0,0,0,0,0,0,0,0
erc20-public-burn,0,0,0,0,0,0,0,0,0,0
accessible-selfdestruct,0,0,0,0,0,0,0,0,0,0
oracle-price-update-not-restricted,0,0,0,0,0,0,0,0,0,0
uniswap-callback-not-protected,0,0,0,0,0,0,0,0,0,0


In [129]:
statistics_matrices['arithmetic']

,divide-before-multiply,basic-arithmetic-underflow,SOLIDITY_DIV_MUL
detector,,,
divide-before-multiply,0,0,0
basic-arithmetic-underflow,0,0,0
SOLIDITY_DIV_MUL,0,0,0


In [130]:
statistics_matrices['shadowing']

,shadowing-local,shadowing-state,shadowing-abstract
detector,,,
shadowing-local,0,0,0
shadowing-state,0,0,0
shadowing-abstract,0,0,0


In [131]:
statistics_matrices['compliance']

,erc20-interface,SOLIDITY_ERC20_TRANSFER_SHOULD_THROW
detector,,
erc20-interface,0,0
SOLIDITY_ERC20_TRANSFER_SHOULD_THROW,0,0


In [132]:
statistics_matrices['compliance']

,erc20-interface,SOLIDITY_ERC20_TRANSFER_SHOULD_THROW
detector,,
erc20-interface,0,0
SOLIDITY_ERC20_TRANSFER_SHOULD_THROW,0,0


In [133]:
statistics_matrices['timestamp']

,timestamp,weak-prng
detector,,
timestamp,0,0
weak-prng,0,0


In [134]:
statistics_matrices['initialization']

,uninitialized-local,uninitialized-state
detector,,
uninitialized-local,0,0
uninitialized-state,0,0


In [135]:
statistics_matrices['poor-logic-flaws']

,incorrect-equality,boolean-cst,incorrect-use-of-blockhash,SOLIDITY_EXACT_TIME,SOLIDITY_BALANCE_EQUALITY
detector,,,,,
incorrect-equality,0,0,0,3,2
boolean-cst,0,0,0,0,0
incorrect-use-of-blockhash,0,0,0,0,0
SOLIDITY_EXACT_TIME,3,0,0,0,0
SOLIDITY_BALANCE_EQUALITY,2,0,0,0,0


In [136]:
statistics_matrices['initialization']

,uninitialized-local,uninitialized-state
detector,,
uninitialized-local,0,0
uninitialized-state,0,0


In [137]:
statistics_matrices['denial-of-service']

,locked-ether,calls-loop,msg-value-loop,SOLIDITY_LOCKED_MONEY,SOLIDITY_TRANSFER_IN_LOOP
detector,,,,,
locked-ether,0,0,0,0,0
calls-loop,0,0,0,0,0
msg-value-loop,0,0,0,0,0
SOLIDITY_LOCKED_MONEY,0,0,0,0,0
SOLIDITY_TRANSFER_IN_LOOP,0,0,0,0,0


In [138]:
statistics_matrices['state-corruption']

,controlled-array-length,SOLIDITY_ARRAY_LENGTH_MANIPULATION
detector,,
controlled-array-length,0,0
SOLIDITY_ARRAY_LENGTH_MANIPULATION,0,0


In [139]:
statistics_matrices['external-call-validation']

,unchecked-transfer,unchecked-lowlevel
detector,,
unchecked-transfer,0,0
unchecked-lowlevel,0,0
